In [ ]:
# no progress since saturday....I have no idea what I'm doing...I started over because all I was doing was looking at someone elses code and copying it.  
#I felt like I needed to reread the slides and understand what is going on

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import datetime
## code to display all columns:
# pd.set_option('display.max_columns', None)


In [2]:
members = pd.read_csv('../data/NSS_GreatCare_Members.csv')
ip_admin = pd.read_csv('../data/NSS_Hospital_Inpatient_Admissions.csv')
office_visits = pd.read_csv('../data/NSS_PrimaryCare_Office_Visits.csv', low_memory = False)

In [3]:
members.head(20)


,Patient ID,Primary Plan ID,Primary Payor ID,Is Patient Alive,Patient Age Integer,Patient Date of Birth,Patient ZIP Code 5-Digit,Count Primary Care Visits in Last 36 Months,Count Visits with Attributee Individual Primary Care Provider i,Count Visits with Attributee Primary Care Provider Group in Las,...,Is CCW Prostate Cancer,Is CCW Rheumatoid Arthritis or Osteoarthritis,Is CCW Stroke or Transient Ischemic Attack,Is CCW ASCVD,Is CCW Anxiety Disorders,Is CCW Depressive Disorders,Is CCW Schizophrenia or Other Psychotic Disorders,Is CCW Bipolar Disorder,Period Start Date,Period End Date
0,DS1|1,DS1|5047W,DS1|1,1,54,1958-11-08,92111.0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,2013-01-01 00:00:00+00,2013-04-11 00:00:00+00
1,DS1|10,DS1|5047W,DS1|1,1,54,1958-12-10,90024.0,6,1.0,5.0,...,0,0,0,0,1,0,0,0,2012-12-10 00:00:00+00,2013-01-01 00:00:00+00
2,DS1|100,DS1|5047W,DS1|1,1,2,2012-08-04,90008.0,1,1.0,1.0,...,0,0,0,0,0,0,0,0,2015-04-18 00:00:00+00,2015-05-18 00:00:00+00
3,DS1|1000,DS1|5047W,DS1|1,1,6,2003-12-25,94131.0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,2010-01-01 00:00:00+00,2010-04-09 00:00:00+00
4,DS1|10000,DS1|5047W,DS1|1,1,7,2005-07-15,90301.0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,2012-08-01 00:00:00+00,2012-10-02 00:00:00+00
5,DS1|10001,DS1|5047X,DS1|1,1,40,1972-11-02,92111.0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,2013-01-01 00:00:00+00,2013-01-30 00:00:00+00
6,DS1|10002,DS1|5047X,DS1|1,1,23,1988-02-04,90064.0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,2012-01-01 00:00:00+00,2012-02-04 00:00:00+00
7,DS1|10003,DS1|5047X,DS1|1,1,24,1991-10-17,91343.0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,2016-05-07 00:00:00+00,2016-10-17 00:00:00+00
8,DS1|10004,DS1|5047X,DS1|1,1,9,2003-08-12,91606.0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,2013-01-01 00:00:00+00,2013-04-12 00:00:00+00
9,DS1|10005,DS1|5047Y,DS1|1,1,55,1957-09-28,94132.0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,2013-01-01 00:00:00+00,2013-09-28 00:00:00+00


In [4]:
#all of the ip_admin code is what Richie has in his notebook..we all worked together on this but I feel like I only did some minor lifting 
#compared to my peers.  
ip_admin['Encounter Start Date'] = pd.to_datetime(ip_admin['Encounter Start Date'], format='%Y-%m-%d')
ip_admin['Encounter End Date'] = pd.to_datetime(ip_admin['Encounter End Date'], format='%Y-%m-%d')

In [5]:
# admissions = admissions[['Encounter ID','Patient ID','Facility Provider NPI','Encounter Start Date','Encounter End Date']]

In [6]:
ip_admin.head()

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,...,1,1,21.0,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,...,1,1,21.0,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,...,1,1,21.0,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,...,1,1,21.0,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0
4,HIA|DS1|10118|DS1|NPI|1992818256|1,DS1|10118,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1518045608,1.518046e+09,1992818256,2015-03-11,2015-03-14,...,2,1,NaN,NaN,NaN,F32.9,"Major depressive disorder, single episode, uns...",0,0.00,0.0


In [7]:
ip_admin = ip_admin.sort_values(['Patient ID','Encounter Start Date', 'Encounter Start Date'], ascending = (False, True, True))

In [8]:
ip_admin = ip_admin.sort_values(['Patient ID','Encounter Start Date', 'Encounter Start Date'], ascending = (False, True, True))

In [9]:
ip_admin1 = ip_admin[['Patient ID', 'Encounter Start Date', 'Encounter End Date']]

In [10]:
ip_admin1

,Patient ID,Encounter Start Date,Encounter End Date
6212,DS1|9996,2010-03-28,2010-03-31
6210,DS1|9994,2015-11-21,2015-11-23
6211,DS1|9994,2015-11-24,2015-11-27
6209,DS1|9981,2014-08-07,2014-08-10
6208,DS1|9979,2015-02-06,2015-02-11
...,...,...,...
3,DS1|10101,2012-12-06,2012-12-09
2,DS1|10073,2012-06-13,2012-06-21
1,DS1|10043,2014-01-05,2014-01-12
0,DS1|10033,2013-01-12,2013-01-14


In [11]:
ip_admin1['Previous End Date'] = (ip_admin1.groupby('Patient ID')['Encounter End Date'].shift(1))
ip_admin1['Days Since Last Visit'] = ip_admin1['Encounter Start Date'] - ip_admin1['Previous End Date']
ip_admin1 = ip_admin1[ip_admin1['Days Since Last Visit'].notnull()]
ip_admin_30 = ip_admin1[ip_admin1['Days Since Last Visit'] <= '30 days']
ip_admin_under_30 = ip_admin1[ip_admin1['Days Since Last Visit'] < '30 days']
ip_admin_30

<ipython-input-11-6cce26c40dd4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ip_admin1['Previous End Date'] = (ip_admin1.groupby('Patient ID')['Encounter End Date'].shift(1))
<ipython-input-11-6cce26c40dd4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ip_admin1['Days Since Last Visit'] = ip_admin1['Encounter Start Date'] - ip_admin1['Previous End Date']


,Patient ID,Encounter Start Date,Encounter End Date,Previous End Date,Days Since Last Visit
6211,DS1|9994,2015-11-24,2015-11-27,2015-11-23,1 days
6202,DS1|9943,2013-10-25,2013-10-28,2013-10-16,9 days
6199,DS1|9935,2011-01-25,2011-01-26,2011-01-24,1 days
6197,DS1|9935,2011-01-26,2011-02-07,2011-01-26,0 days
6185,DS1|9809,2013-01-20,2013-01-24,2013-01-11,9 days
...,...,...,...,...,...
38,DS1|10451,2013-05-01,2013-05-09,2013-04-26,5 days
40,DS1|10451,2013-07-26,2013-08-03,2013-07-01,25 days
41,DS1|10451,2013-08-31,2013-09-02,2013-08-03,28 days
43,DS1|10451,2013-12-17,2013-12-19,2013-11-20,27 days


In [12]:

rows = len(ip_admin_30)
rows

755

In [13]:
ip_admin_30['Patient ID'].nunique()

479

In [14]:
ad_count = len(ip_admin)

In [15]:
ip_admin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6213 entries, 6212 to 117
Data columns (total 23 columns):
 #   Column                                               Non-Null Count  Dtype         
---  ------                                               --------------  -----         
 0   Encounter ID                                         6213 non-null   object        
 1   Patient ID                                           6213 non-null   object        
 2   Encounter Primary Payor ID                           6213 non-null   object        
 3   Encounter Primary Plan ID                            6213 non-null   object        
 4   Facility Provider ID                                 6213 non-null   object        
 5   Attending Provider ID                                5884 non-null   object        
 6   Attending Provider NPI                               5884 non-null   float64       
 7   Facility Provider NPI                                6213 non-null   object        
 

In [16]:
ip_admin1[ip_admin1['Days Since Last Visit'].notnull()]


,Patient ID,Encounter Start Date,Encounter End Date,Previous End Date,Days Since Last Visit
6211,DS1|9994,2015-11-24,2015-11-27,2015-11-23,1 days
6202,DS1|9943,2013-10-25,2013-10-28,2013-10-16,9 days
6203,DS1|9943,2014-10-28,2014-10-30,2013-10-28,365 days
6199,DS1|9935,2011-01-25,2011-01-26,2011-01-24,1 days
6197,DS1|9935,2011-01-26,2011-02-07,2011-01-26,0 days
...,...,...,...,...,...
36,DS1|10451,2014-06-29,2014-06-29,2014-05-29,31 days
21,DS1|10291,2011-02-24,2011-03-05,2010-05-15,285 days
18,DS1|10253,2010-06-02,2010-06-06,2010-04-16,47 days
17,DS1|10253,2011-10-15,2011-11-13,2010-06-06,496 days


In [17]:
# read_rate = round(read_count/ad_count *100,2)
# read_rate


In [18]:
#admissions = admissions[['Patient ID', 'Encounter Start Date','Encounter End Date']]

In [19]:
# admissions_grouped = admissions.groupby(['Patient ID','Encounter Start Date'],as_index = False).sum()

In [20]:
# admissions_grouped

In [21]:
# admissions.info()  

In [22]:
# admissions[['First','second','third','fourth','fifth','sixth','Last']] = admissions['Encounter ID'].apply(
#    lambda x: pd.Series(str(x).split("|")))
# admissions.head()

In [23]:
office_visits.head()

,Encounter ID,Patient ID,Service Provider ID,Attending Provider ID,Provider Group Provider ID,Facility Provider ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Attending Provider NPI,Facility Provider NPI,Service Provider NPI,Provider Group NPI,Encounter Date,Is Qualifying Preventive Primary Care Service,Is Qualifying Primary Care Service Provider,Is Qualifying Primary Care Attending,Is Qualifying Primary Care Provider Group,Qualifying Preventive Primary Care HCPCS Code
0,COV|EE|DS1|10076|20110225|DS1|NPI|1487856779,DS1|10076,DS1|NPI|1487856779,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1.487857e+09,1.043427e+09,2011-02-18,0,1,0,0,NaN
1,COV|EE|DS1|10088|20140222|DS1|NPI|1528030996,DS1|10088,DS1|NPI|1528030996,DS1|NPI|1528030996,DS1|NPI|1043427248,DS1|NPI|1902803315,DS1|1,DS1|5047W,1.528031e+09,1.902803e+09,1.528031e+09,1.043427e+09,2014-02-22,0,1,1,1,NaN
2,COV|EE|DS1|10165|20140907|DS1|NPI|1477579365,DS1|10165,DS1|NPI|1477579365,NaN,DS1|NPI|1043427248,NaN,DS1|1,DS1|5047W,NaN,NaN,1.477579e+09,1.043427e+09,2014-08-31,0,1,0,0,NaN
3,COV|EE|DS1|10240|20130624|DS1|NPI|1821178088,DS1|10240,DS1|NPI|1821178088,DS1|NPI|1821178088,DS1|NPI|1407058787,DS1|NPI|1104842020,DS1|1,DS1|5047W,1.821178e+09,1.104842e+09,1.821178e+09,1.407059e+09,2013-06-17,0,1,1,1,NaN
4,COV|EE|DS1|10293|20160904|DS1|NPI|1609004191,DS1|10293,DS1|NPI|1609004191,NaN,DS1|NPI|1184722779,NaN,DS1|1,DS1|5047X,NaN,NaN,1.609004e+09,1.184723e+09,2016-09-04,0,1,0,0,NaN


In [24]:
office_visits2 = office_visits[['Patient ID','Encounter Date']]
office_visits2

,Patient ID,Encounter Date
0,DS1|10076,2011-02-18
1,DS1|10088,2014-02-22
2,DS1|10165,2014-08-31
3,DS1|10240,2013-06-17
4,DS1|10293,2016-09-04
...,...,...
231368,DS1|9999,2015-02-10
231369,DS1|99,2011-08-02
231370,DS1|99,2011-04-13
231371,DS1|9,2014-04-07


In [25]:
pcp_post_discharge = pd.merge(ip_admin1, office_visits2, on ='Patient ID')
pcp_post_discharge

,Patient ID,Encounter Start Date,Encounter End Date,Previous End Date,Days Since Last Visit,Encounter Date
0,DS1|9994,2015-11-24,2015-11-27,2015-11-23,1 days,2015-05-03
1,DS1|9994,2015-11-24,2015-11-27,2015-11-23,1 days,2014-04-05
2,DS1|9994,2015-11-24,2015-11-27,2015-11-23,1 days,2014-07-19
3,DS1|9994,2015-11-24,2015-11-27,2015-11-23,1 days,2015-06-22
4,DS1|9994,2015-11-24,2015-11-27,2015-11-23,1 days,2015-12-05
...,...,...,...,...,...,...
14039,DS1|10291,2011-02-24,2011-03-05,2010-05-15,285 days,2010-05-28
14040,DS1|10253,2010-06-02,2010-06-06,2010-04-16,47 days,2009-12-19
14041,DS1|10253,2010-06-02,2010-06-06,2010-04-16,47 days,2010-04-24
14042,DS1|10253,2011-10-15,2011-11-13,2010-06-06,496 days,2009-12-19


In [26]:
#apply date_time to office_visits

office_visits['Encounter Date'] = pd.to_datetime(office_visits['Encounter Date'], format='%Y-%m-%d')


#make df and merge

ad_cut = ip_admin[['Patient ID','Encounter End Date']]
pcp_cut = office_visits[['Patient ID','Encounter Date']]
two = ad_cut.merge(pcp_cut, left_on ='Patient ID', right_on = 'Patient ID',how='left', indicator = False)
#sort
two = two.sort_values(['Patient ID',"Encounter Date", "Encounter End Date"], ascending = (False, True, True))
#sub encounter date from end dates
two['Days Since Last Visit'] = two['Encounter Date'] - two['Encounter End Date']
two_top = two[two['Days Since Last Visit'] <= '7 days']
two_top = two_top[two_top['Days Since Last Visit'] >= '0 days']
two_top


,Patient ID,Encounter End Date,Encounter Date,Days Since Last Visit
18,DS1|9981,2014-08-10,2014-08-11,1 days
21,DS1|9981,2014-08-10,2014-08-12,2 days
25,DS1|9979,2015-02-11,2015-02-15,4 days
39,DS1|9978,2015-09-05,2015-09-11,6 days
51,DS1|9945,2013-02-11,2013-02-14,3 days
...,...,...,...,...
43852,DS1|10303,2012-06-24,2012-06-28,4 days
43845,DS1|10303,2012-06-24,2012-06-30,6 days
44042,DS1|10101,2012-12-09,2012-12-12,3 days
44052,DS1|10073,2012-06-21,2012-06-27,6 days


In [27]:
#  The math is not correct here...
#make df and merge
ad_cut = ip_admin1[['Patient ID','Encounter End Date']]
pcp_cut = office_visits[['Patient ID','Encounter Date']]
two = ad_cut.merge(pcp_cut, left_on ='Patient ID', right_on = 'Patient ID',how='left', indicator = False)
#sub encounter date from end dates
two['Days Since Last Visit'] = two['Encounter Date'] - two['Encounter End Date']
two_top = two[two['Days Since Last Visit'] <= '7 days']
two_top = two_top[two_top['Days Since Last Visit'] >= '0 days']
#count
count_two_top = len(two_top)
two_top
#find the pcp visit within 7 days rate
pcp_visit_rate = round(count_two_top/ad_count *100,2)
pcp_visit_rate


6.78